In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('../raw_data/df_grouped_rate.csv') #폴더 위치는 상이할 수 있음

In [2]:
y=df.iloc[:,3:4]
X= df.iloc[:,1:].drop(['sales_total'],axis=1)
X.head(3)

,district,code,s_store_no_of_store,s_store_no_of_opening,s_store_no_of_closing,s_work_female,s_float_male,s_float_female,b_facil_total,b_apt_avg_price,b_income_avg_monthly_inc,sales_weekday,sales_female,sales_2030s,sales_06_11,sales_11_14,sales_14_17,sales_17_21,sales_21_24
0,1000001,CS100001,16.5,2,2,1748,211158,145498,129.0,188530154.0,3889111.0,0.707812,0.318123,0.341417,0.007379,0.344443,0.164275,0.345822,0.120878
1,1000001,CS100002,2.0,1,1,1748,211158,145498,129.0,188530154.0,3889111.0,0.734836,0.340414,0.468521,0.006540,0.447948,0.173301,0.363921,0.008290
2,1000001,CS100003,2.0,0,0,1748,211158,145498,129.0,188530154.0,3889111.0,0.914956,0.321958,0.480098,0.000000,0.524393,0.057219,0.227561,0.182710


In [3]:
# 더미변수화
X_dum1 = pd.get_dummies(X.iloc[:,0])  #district는 범주형으로 인식 안해서 따로 실시
X_dum2 = pd.get_dummies(X.iloc[:,1:])
X_dum = pd.concat([X_dum1, X_dum2],axis=1)


### 회귀분석 (1010개 상권코드)

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_dum, y, test_size=0.2) 

# 로버스트 스케일링
rb = RobustScaler()
Xs_train = rb.fit_transform(X_train)
Xs_test = rb.transform(X_test)

# 회귀분석
regressor = LinearRegression()
model = regressor.fit(Xs_train, y_train)

In [5]:
# 정확도 결과값
print(model.score(Xs_train, y_train))
print(model.score(Xs_test, y_test))

print('='*50)
print('='*50)
# 오차 결과값
from sklearn import metrics
y_pred = model.predict(Xs_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

0.4429290720507544
0.42902697001808043
Mean Absolute Error: 650471471.7366794
Mean Squared Error: 1.6228396668654003e+18
Root Mean Squared Error: 1273907244.2157633


In [ ]:
#OLS summary를 보기위해 
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_dum, y, test_size=0.2, random_state=0) # y 값이 정의되지 않아 df.sales_total 으로 대체하겠습니다.
# 로버스트 스케일링
rb = RobustScaler()
Xs_train = rb.fit_transform(X_train)
Xs_test = rb.transform(X_test)

model = sm.OLS(y_train, Xs_train)
result = model.fit()
print(result.summary())

```
  OLS Regression Results                            
==============================================================================
Dep. Variable:            sales_total   R-squared:                       0.443
Model:                            OLS   Adj. R-squared:                  0.436
Method:                 Least Squares   F-statistic:                     66.62
Date:                Sat, 15 Feb 2020   Prob (F-statistic):               0.00
Time:                        16:13:53   Log-Likelihood:            -2.0276e+06
No. Observations:               90612   AIC:                         4.057e+06
Df Residuals:                   89544   BIC:                         4.067e+06
Df Model:                        1067                                         
Covariance Type:            nonrobust                                         
==============================================================================
```

```
==============================================================================
Omnibus:                   116172.839   Durbin-Watson:                   2.007
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         85264506.370
Skew:                           6.589   Prob(JB):                         0.00
Kurtosis:                     152.700   Cond. No.                     1.25e+17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 4.28e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
```


### 결론) 1010개의 상권코드와 서비스코드를 원핫인코딩 하면 다중공선성이 발생한다

In [26]:
# Cross Validation 실시(10회)
from sklearn.model_selection import cross_val_score, cross_validate, cross_val_predict

scores = cross_val_score(model, X_dum, y,scoring='neg_mean_squared_error',n_jobs=-1,cv=10)
print(scores.mean())
scores

-5.1939627337549884e+23


array([-1.49316595e+18, -1.77724911e+18, -1.48891493e+18, -1.72991690e+18,
       -1.61632030e+18, -2.06310464e+18, -1.94553029e+18, -2.28336208e+18,
       -4.54186243e+24, -6.52085910e+23])

In [7]:
import sklearn
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'accuracy', 'roc_auc', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'brier_score_loss', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])

In [8]:
from sklearn.metrics import r2_score
R2_lr = []
MAE_lr = []
MSE_lr = []
RMSE_lr = []
for i in range(10):
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_dum, y, test_size=0.2) 

    # 로버스트 스케일링
    rb = RobustScaler()
    Xs_train = rb.fit_transform(X_train)
    Xs_test = rb.transform(X_test)

    # 회귀분석
    regressor = LinearRegression()
    model = regressor.fit(Xs_train, y_train)

    print(i+1,"번째 시도")
    # 정확도 결과값
    print(model.score(Xs_train, y_train))
    print(model.score(Xs_test, y_test))

    # 오차 결과값
    y_pred = model.predict(Xs_test)
    R2_lr.append(r2_score(y_test , y_pred))
    MAE_lr.append(metrics.mean_absolute_error(y_test, y_pred))
    MSE_lr.append(metrics.mean_squared_error(y_test, y_pred))
    RMSE_lr.append(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print('='*50)
    print('='*50)

1 번째 시도
0.43295550795416243
0.46764540986097264
2 번째 시도
0.4512744905536025
0.40246816411536745
3 번째 시도
0.44184473285831516
0.4326529746566693
4 번째 시도
0.4456531746312221
0.4127726056361403
5 번째 시도
0.43692286889880483
0.4533253942535944
6 번째 시도
0.4449587150800643
0.41997169592785366
7 번째 시도
0.44422570807458217
0.42474832400608553
8 번째 시도
0.43851343442507795
0.4433980244749468
9 번째 시도
0.438934419310843
0.4467996451486497
10 번째 시도
0.4481330335904439
0.4100426070548917


In [9]:
print(R2_lr)
print(MAE_lr)
print(MSE_lr)
print(RMSE_lr)

[0.4676454098609726, 0.40246816411536745, 0.4326529746566693, 0.41277260563614027, 0.4533253942535944, 0.41997169592785366, 0.42474832400608553, 0.4433980244749468, 0.4467996451486498, 0.4100426070548917]
[647082035.7392398, 648679503.1301373, 644071487.905134, 634007679.8162274, 638795168.7209641, 648808333.2329934, 634857412.621154, 655537567.7177416, 641805331.6786739, 652846556.0486028]
[1.5801062506696507e+18, 1.9052248014590863e+18, 1.543910135309669e+18, 1.405659935501998e+18, 1.6057576388806254e+18, 1.5598905701956685e+18, 1.6231874379158515e+18, 1.6300242525811876e+18, 1.450448607159404e+18, 1.7310374790547484e+18]
[1257022772.5342333, 1380298808.7581205, 1242541804.2503314, 1185605303.421842, 1267184926.8676713, 1248955791.9300702, 1274043734.695105, 1276724031.4888678, 1204345717.4579914, 1315688975.0449185]


In [10]:
import pickle
with open('R2_lr.pkl', 'wb') as f:
    pickle.dump(R2_lr, f) #피클파일 저장
with open('MAE_lr.pkl', 'wb') as f:
    pickle.dump(MAE_lr, f) #피클파일 저장
with open('MSE_lr.pkl', 'wb') as f:
    pickle.dump(MSE_lr, f) #피클파일 저장

with open('RMSE_lr.pkl', 'wb') as f:
    pickle.dump(RMSE_lr, f) #피클파일 저장



In [13]:
#load
with open('R2_lr.pkl', 'rb') as f:
    data = pickle.load(f) # 단 한줄씩 읽어옴

## 시군구 OLS

In [16]:
df = pd.read_csv("../raw_data/df_grouped_rate.csv", encoding ='utf-8')
cgoongu = pd.read_csv("../raw_data/cgoongoo1.csv", encoding ='utf-8')
df_merged = pd.merge(df, cgoongu, on=['district'])
df_merged.drop(['district','year'], axis=1, inplace =True)

In [17]:
y=df_merged.iloc[:,1:2]
X= df_merged.drop(['sales_total'],axis=1)
X.head(3)

,code,s_store_no_of_store,s_store_no_of_opening,s_store_no_of_closing,s_work_female,s_float_male,s_float_female,b_facil_total,b_apt_avg_price,b_income_avg_monthly_inc,sales_weekday,sales_female,sales_2030s,sales_06_11,sales_11_14,sales_14_17,sales_17_21,sales_21_24,cgoongoo
0,CS100001,16.5,2,2,1748,211158,145498,129.0,188530154.0,3889111.0,0.707812,0.318123,0.341417,0.007379,0.344443,0.164275,0.345822,0.120878,11110
1,CS100002,2.0,1,1,1748,211158,145498,129.0,188530154.0,3889111.0,0.734836,0.340414,0.468521,0.006540,0.447948,0.173301,0.363921,0.008290,11110
2,CS100003,2.0,0,0,1748,211158,145498,129.0,188530154.0,3889111.0,0.914956,0.321958,0.480098,0.000000,0.524393,0.057219,0.227561,0.182710,11110


In [18]:
# 더미변수화
X_dum1 = pd.get_dummies(X.iloc[:,0:-1])  #district는 범주형으로 인식 안해서 따로 실시
X_dum2 = pd.get_dummies(X.iloc[:,-1])
X_dum = pd.concat([X_dum1, X_dum2],axis=1)
X_dum.head(3)

,s_store_no_of_store,s_store_no_of_opening,s_store_no_of_closing,s_work_female,s_float_male,s_float_female,b_facil_total,b_apt_avg_price,b_income_avg_monthly_inc,sales_weekday,...,11500,11530,11545,11560,11590,11620,11650,11680,11710,11740
0,16.5,2,2,1748,211158,145498,129.0,188530154.0,3889111.0,0.707812,...,0,0,0,0,0,0,0,0,0,0
1,2.0,1,1,1748,211158,145498,129.0,188530154.0,3889111.0,0.734836,...,0,0,0,0,0,0,0,0,0,0
2,2.0,0,0,1748,211158,145498,129.0,188530154.0,3889111.0,0.914956,...,0,0,0,0,0,0,0,0,0,0


In [53]:
#OLS summary를 보기위해 
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_dum, y, test_size=0.2, random_state=0) # y 값이 정의되지 않아 df.sales_total 으로 대체하겠습니다.
# 로버스트 스케일링
rb = RobustScaler()
Xs_train = rb.fit_transform(X_train)
Xs_test = rb.transform(X_test)

model = sm.OLS(y_train, Xs_train)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:            sales_total   R-squared:                       0.405
Model:                            OLS   Adj. R-squared:                  0.405
Method:                 Least Squares   F-statistic:                     725.3
Date:                Thu, 20 Feb 2020   Prob (F-statistic):               0.00
Time:                        19:13:06   Log-Likelihood:            -2.0306e+06
No. Observations:               90612   AIC:                         4.061e+06
Df Residuals:                   90526   BIC:                         4.062e+06
Df Model:                          85                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          5.418e+08   5.01e+06    108.138      0.0

In [20]:
R2_cgoongu = []
MAE_cgoongu = []
MSE_cgoongu = []
RMSE_cgoongu = []
for i in range(10):
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_dum, y, test_size=0.2) 

    # 로버스트 스케일링
    rb = RobustScaler()
    Xs_train = rb.fit_transform(X_train)
    Xs_test = rb.transform(X_test)

    # 회귀분석
    regressor = LinearRegression()
    model = regressor.fit(Xs_train, y_train)

    print(i+1,"번째 시도")
    # 정확도 결과값
    print(model.score(Xs_train, y_train))
    print(model.score(Xs_test, y_test))

    # 오차 결과값
    y_pred = model.predict(Xs_test)
    R2_cgoongu.append(r2_score(y_test , y_pred))
    MAE_cgoongu.append(metrics.mean_absolute_error(y_test, y_pred))
    MSE_cgoongu.append(metrics.mean_squared_error(y_test, y_pred))
    RMSE_cgoongu.append(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print('='*50)
    print('='*50)

1 번째 시도
0.40101294886460426
0.4246287073118932
2 번째 시도
0.40387559472256784
0.4142720872316741
3 번째 시도
0.4009605327891068
0.42279858858453434
4 번째 시도
0.41490993936264114
0.3765957047833143
5 번째 시도
0.4120039061023595
0.38211957042829525
6 번째 시도
0.41222954124161665
0.3818497417394955
7 번째 시도
0.3981734032249744
0.44034405926548686
8 번째 시도
0.4113814075660268
0.3831707121552382
9 번째 시도
0.39652339589276875
0.44112163543639
10 번째 시도
0.40886735641160277
0.39457707533939446


In [21]:
print(R2_cgoongu) 
print(MAE_cgoongu) 
print(MSE_cgoongu) 
print(RMSE_cgoongu) 

[0.4246287073118933, 0.4142720872316741, 0.42279858858453434, 0.3765957047833143, 0.38211957042829525, 0.3818497417394955, 0.4403440592654868, 0.38317071215523824, 0.44112163543639005, 0.39457707533939446]
[628049169.359246, 632668749.8601952, 636877598.5640312, 645958415.2668079, 630451027.1724716, 638788512.969717, 624317006.7914625, 618851934.5385159, 633982518.8671257, 633873525.0161568]
[1.66570200071061e+18, 1.5949846785149663e+18, 1.7695274067194458e+18, 2.1208731276893676e+18, 1.812318596794043e+18, 1.8304803614836134e+18, 1.454388182620031e+18, 1.7278271833336384e+18, 1.6316103373000412e+18, 1.8020150414890463e+18]
[1290620781.140072, 1262927028.1829295, 1330235846.276684, 1456321780.270201, 1346223828.6384783, 1352952460.9104393, 1205980175.0526545, 1314468403.3226657, 1277345034.5541103, 1342391538.0726469]


In [22]:
import pickle
with open('R2_cgoongu.pkl', 'wb') as f:
    pickle.dump(R2_cgoongu, f) #피클파일 저장
with open('MAE_cgoongu.pkl', 'wb') as f:
    pickle.dump(MAE_lr, f) #피클파일 저장
with open('MSE_cgoongu.pkl', 'wb') as f:
    pickle.dump(MSE_cgoongu, f) #피클파일 저장

with open('RMSE_cgoongu.pkl', 'wb') as f:
    pickle.dump(RMSE_cgoongu, f) #피클파일 저장




### Random Forest (상권코드 1010개)

In [23]:
import pandas as pd 
df = pd.read_csv('../raw_data/df_grouped_rate.csv') #폴더 위치는 상이할 수 있음
y=df.iloc[:,3:4]
X= df.iloc[:,1:].drop(['sales_total'],axis=1)
# 더미변수화
X_dum1 = pd.get_dummies(X.iloc[:,0])  #district는 범주형으로 인식 안해서 따로 실시
X_dum2 = pd.get_dummies(X.iloc[:,1:])
X = pd.concat([X_dum1, X_dum2],axis=1)

In [24]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
R2_RF = []
MAE_RF = []
MSE_RF = []
RMSE_RF = []
for i in range(10):
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    Xs_train = X_train
    Xs_test = X_test
    regr = RandomForestRegressor(n_estimators=50,max_depth=20, n_jobs=-1)
    model=regr.fit(Xs_train, y_train)
    y_pred=model.predict(Xs_test)
    
    print(i+1,"번째 시도")
    # 정확도 결과값
    print(model.score(Xs_train, y_train))
    print(model.score(Xs_test, y_test))

    # 오차 결과값
    R2_RF.append(r2_score(y_test , y_pred))
    MAE_RF.append(metrics.mean_absolute_error(y_test, y_pred))
    MSE_RF.append(metrics.mean_squared_error(y_test, y_pred))
    RMSE_RF.append(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print('='*50)
    print('='*50)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


1 번째 시도
0.8855539537969531
0.7123008661778889


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


2 번째 시도
0.8877133339202862
0.7000830569701366


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


...

8 번째 시도
0.8800700554929304
0.6994841394350301


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


9 번째 시도
0.8871357493317835
0.7142729852698322


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


10 번째 시도
0.8884251082730303
0.6934375074159209


In [25]:
print(R2_RF) 
print(MAE_RF) 
print(MSE_RF) 
print(RMSE_RF) 

[0.7123008661778889, 0.7000830569701366, 0.7292598113397541, 0.7191962331516705, 0.692337388176013, 0.7107033500590043, 0.7135588304672187, 0.6994841394350301, 0.7142729852698322, 0.6934375074159209]
[449785112.6294148, 446212734.7671297, 447890754.86098444, 451625667.5133701, 445865890.12949145, 442355273.1377831, 446089670.0951218, 447415163.7190714, 445434133.4830528, 437294514.8725618]
[8.265276739061085e+17, 8.120435978355119e+17, 8.2239566534996e+17, 8.091070983962254e+17, 8.547321165231905e+17, 8.261774271114982e+17, 8.443124401772238e+17, 9.201507632837652e+17, 8.038788677680585e+17, 8.06667925760289e+17]
[909135674.0916663, 901134616.9332925, 906860333.9820085, 899503806.7713919, 924517234.302958, 908943027.4288363, 918864756.1949602, 959244892.2375168, 896592921.9930629, 898146939.9604326]


In [26]:
import pickle
with open('pkl_files/R2_RF.pkl', 'wb') as f:
    pickle.dump(R2_RF, f) #피클파일 저장
with open('pkl_files/MAE_RF.pkl', 'wb') as f:
    pickle.dump(MAE_lr, f) #피클파일 저장
with open('pkl_files/MSE_RF.pkl', 'wb') as f:
    pickle.dump(MSE_RF, f) #피클파일 저장

with open('pkl_files/RMSE_RF.pkl', 'wb') as f:
    pickle.dump(RMSE_RF, f) #피클파일 저장


## Random Forest (시군구)

In [27]:
import pandas as pd

df = pd.read_csv("../raw_data/df_grouped_rate.csv", encoding ='utf-8')
cgoongu = pd.read_csv("../raw_data/cgoongoo1.csv", encoding ='utf-8')
df_merged = pd.merge(df, cgoongu, on=['district'])
df_merged.drop(['district','year'], axis=1, inplace =True)

In [28]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
y=df_merged.iloc[:,1:2]
X= df_merged.drop(['sales_total'],axis=1)
# X.columns
# 더미변수화
X_dum1 = pd.get_dummies(X.iloc[:,0:-1])  #district는 범주형으로 인식 안해서 따로 실시
X_dum2 = pd.get_dummies(X.iloc[:,-1])
X = pd.concat([X_dum1, X_dum2],axis=1)
# X_dum.columns

In [29]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.metrics import r2_score
import numpy as np 

R2_RF_c = []
MAE_RF_c = []
MSE_RF_c = []
RMSE_RF_c = []
for i in range(10):
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    Xs_train = X_train
    Xs_test = X_test
    regr = RandomForestRegressor(n_estimators=50,max_depth=20, n_jobs=-1)
    model=regr.fit(Xs_train, y_train)
    y_pred=model.predict(Xs_test)
    
    print(i+1,"번째 시도")
    # 정확도 결과값
    print(model.score(Xs_train, y_train))
    print(model.score(Xs_test, y_test))

    # 오차 결과값
    R2_RF_c.append(r2_score(y_test , y_pred))
    MAE_RF_c.append(metrics.mean_absolute_error(y_test, y_pred))
    MSE_RF_c.append(metrics.mean_squared_error(y_test, y_pred))
    RMSE_RF_c.append(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print('='*50)
    print('='*50)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


1 번째 시도
0.8998467497578891
0.6802226302816198


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


2 번째 시도
0.9020416064994576
0.6683817661895358


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


...

8 번째 시도
0.8994470332253633
0.6878557999495694


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


9 번째 시도
0.9051449096029229
0.6593712829737952


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


10 번째 시도
0.9015410527355165
0.6784010416389705


In [30]:
print(R2_RF_c) 
print(MAE_RF_c) 
print(MSE_RF_c) 
print(RMSE_RF_c) 

[0.6802226302816198, 0.6683817661895358, 0.6774419178919397, 0.6622113532206795, 0.6523103043284098, 0.6716634130220733, 0.6578239046395441, 0.6878557999495694, 0.6593712829737952, 0.6784010416389705]
[454503927.4785681, 455461946.7784578, 456022726.0518362, 459388121.43013835, 453582089.42480946, 457196495.6549004, 451462806.02449113, 457738810.23634917, 456390577.3145367, 464367556.78221273]
[9.064740617367328e+17, 9.338313240838047e+17, 9.988792121759087e+17, 1.0294519119889932e+18, 9.961150129492637e+17, 8.98943692568813e+17, 9.280117372720316e+17, 9.124766364279164e+17, 8.606323753026831e+17, 9.081914697825079e+17]
[952089313.9494492, 966349483.4084637, 999439448.9792309, 1014619096.9960072, 998055616.1603739, 948126411.7030034, 963333658.330296, 955236429.5963154, 927702740.8080043, 952990802.5697352]


In [31]:
import pickle
with open('pkl_files/R2_RF_c.pkl', 'wb') as f:
    pickle.dump(R2_RF_c, f) #피클파일 저장
with open('pkl_files/MAE_RF_c.pkl', 'wb') as f:
    pickle.dump(MAE_RF_c, f) #피클파일 저장
with open('pkl_files/MSE_RF_c.pkl', 'wb') as f:
    pickle.dump(MSE_RF_c, f) #피클파일 저장

with open('pkl_files/MSE_RF_c.pkl', 'wb') as f:
    pickle.dump(RMSE_RF_c, f) #피클파일 저장

### Random Fores dummy 변수?

## LightGBM (1010)

In [32]:
import pandas as pd 
df = pd.read_csv('../raw_data/df_grouped_rate.csv') #폴더 위치는 상이할 수 있음
y=df.iloc[:,3:4]
X= df.iloc[:,1:].drop(['sales_total'],axis=1)
# 더미변수화
X_dum1 = pd.get_dummies(X.iloc[:,0])  #district는 범주형으로 인식 안해서 따로 실시
X_dum2 = pd.get_dummies(X.iloc[:,1:])
X = pd.concat([X_dum1, X_dum2],axis=1)

In [33]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
from sklearn.model_selection import train_test_split
R2_lgb_gbdt = []
MAE_lgb_gbdt = []
MSE_lgb_gbdt = []
RMSE_lgb_gbdt = []
for _ in range(10):
    train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=0)
    train_ds = lgb.Dataset(train_x, label = train_y) 
    test_ds = lgb.Dataset(test_x, label = test_y) 

    params = {'learning_rate': 0.01, 
              'max_depth': 16, 
              'boosting': 'gbdt', 
              'objective': 'regression', 
              'metric': 'mse', 
              'is_training_metric': True, 
              'num_leaves': 144, 
              'feature_fraction': 0.9, 
              'bagging_fraction': 0.7, 
              'bagging_freq': 5, 
              'device': 'gpu'
             }
    print(i+1,"번째 시도")
    # 정확도 결과값
    print(model.score(Xs_train, y_train))
    print(model.score(Xs_test, y_test))

    # 오차 결과값
    R2_lgb_gbdt.append(r2_score(y_test , y_pred))
    MAE_lgb_gbdt.append(metrics.mean_absolute_error(y_test, y_pred))
    MSE_lgb_gbdt.append(metrics.mean_squared_error(y_test, y_pred))
    RMSE_lgb_gbdt.append(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print('='*50)
    print('='*50)


10 번째 시도
0.9015410527355165
0.6784010416389705
10 번째 시도
0.9015410527355165
0.6784010416389705
10 번째 시도
0.9015410527355165
0.6784010416389705
10 번째 시도
0.9015410527355165
0.6784010416389705
10 번째 시도
0.9015410527355165
0.6784010416389705
10 번째 시도
0.9015410527355165
0.6784010416389705
10 번째 시도
0.9015410527355165
0.6784010416389705
10 번째 시도
0.9015410527355165
0.6784010416389705
10 번째 시도
0.9015410527355165
0.6784010416389705
10 번째 시도
0.9015410527355165
0.6784010416389705


In [34]:
print(R2_lgb_gbdt) 
print(MAE_lgb_gbdt) 
print(MSE_lgb_gbdt) 
print(RMSE_lgb_gbdt) 

[0.6784010416389705, 0.6784010416389705, 0.6784010416389705, 0.6784010416389705, 0.6784010416389705, 0.6784010416389705, 0.6784010416389705, 0.6784010416389705, 0.6784010416389705, 0.6784010416389705]
[464367556.78221273, 464367556.78221273, 464367556.78221273, 464367556.78221273, 464367556.78221273, 464367556.78221273, 464367556.78221273, 464367556.78221273, 464367556.78221273, 464367556.78221273]
[9.081914697825079e+17, 9.081914697825079e+17, 9.081914697825079e+17, 9.081914697825079e+17, 9.081914697825079e+17, 9.081914697825079e+17, 9.081914697825079e+17, 9.081914697825079e+17, 9.081914697825079e+17, 9.081914697825079e+17]
[952990802.5697352, 952990802.5697352, 952990802.5697352, 952990802.5697352, 952990802.5697352, 952990802.5697352, 952990802.5697352, 952990802.5697352, 952990802.5697352, 952990802.5697352]


In [35]:
import pickle
with open('pkl_files/R2_lgb_gbdt.pkl', 'wb') as f:
    pickle.dump(R2_lgb_gbdt, f) #피클파일 저장
with open('pkl_files/MAE_lgb_gbdt.pkl', 'wb') as f:
    pickle.dump(MAE_lgb_gbdt, f) #피클파일 저장
with open('pkl_files/MSE_lgb_gbdt.pkl', 'wb') as f:
    pickle.dump(MSE_lgb_gbdt, f) #피클파일 저장

with open('pkl_files/RMSE_lgb_gbdt.pkl', 'wb') as f:
    pickle.dump(RMSE_lgb_gbdt, f) #피클파일 저장

## combine predictors using stacking
y_pred, y_test

https://scikit-learn.org/stable/auto_examples/ensemble/plot_stack_predictors.html#sphx-glr-auto-examples-ensemble-plot-stack-predictors-py